In [ ]:
import threading
import time

##Initializing semaphores

sem_barber = threading.Semaphore()   #barber is cutting the hair
sem_customer = threading.Semaphore()   #new customer has entered in the waiting room
sem_leave = threading.Semaphore()   #no entry in shop i.e.,leaves
me = threading.Semaphore()   #mutual exclusion

#inputs

total_waiting_chair = int(input("Enter the total number of chairs: ")) 
occupied_waiting_chair = int(input("Enter the number of chairs used: ")) 
barber_chair = input("Enter the status of barber's chair: ")#when True, chair is occupied
print("Total chairs in the waiting room: ",total_waiting_chair)  
print("Occupied chairs in the waiting room: ",occupied_waiting_chair)  
print("Barber Chair: ",barber_chair) 

barber_sleep=0 #0 means barber is awake and 1 means sleep

def customer():
    global occupied_waiting_chair
    
    while occupied_waiting_chair<=total_waiting_chair:
        if (barber_chair==True and occupied_waiting_chair < total_waiting_chair):
            occupied_waiting_chair+=1
            time.sleep(5) #s
            sem_customer.release()   #semsignal(sem_customer)
            print("%s released lock"%(threading.current_thread().name))
            print("semaphore sem_customer value: ",sem_customer._value)
            time.sleep(5)
            print("Occupied  Chairs are: ",occupied_waiting_chair)
        elif (occupied_waiting_chair==total_waiting_chair and barber_chair==True):
            sem_leave.release()      #Semsignal(sem_leave)
            print("%s released lock"%(threading.current_thread().name))
            print("semaphore sem_leave value: ",sem_leave._value)
            balk()
            me.release()
            print("%s released lock"%(threading.current_thread().name))
            print("semaphore me value: ",me._value)
            break
        else:
            sem_barber.release() #Semsignal(sem_barber) +1
            print("%s released lock"%(threading.current_thread().name))
            print("semaphore sem_barber value: ",sem_barber._value)
            get_haircut()
            break

def cut_hair():
    global barber_chair
    if(barber_chair==True):
       print("Barber's chair is occupied!")
   
    elif(barber_chair==False):
       print("Barber's chair is empty!")

def get_haircut():
    global occupied_waiting_chair
    global barber_chair
    sem_barber.acquire() #semWait(sem_barber) -1
    print("%s acquired lock"%(threading.current_thread().name))
    print("semaphore sem_barber value: ",sem_barber._value)
    barber_chair=True
    if(barber_chair==True and occupied_waiting_chair==0):
        pass
    else:
        print("Barber's chair status :",barber_chair)
    me.acquire() #semWait(me)
    sem_customer.acquire() #semWait(sem_customer)
    if(occupied_waiting_chair==0):
        pass
    else:
        occupied_waiting_chair-=1
    print("%s acquired lock"%(threading.current_thread().name))
    print("semaphore me value: ",me._value)
    print("semaphore sem_customer value: ",sem_customer._value)
    print("Occupied Chairs are: ",occupied_waiting_chair)
    customer()


def barber():
    if(occupied_waiting_chair!=0 and barber_chair==False):
        barber_sleep=0
        cut_hair()
    elif(barber_chair==True):
        pass
    elif(barber_chair==False and occupied_waiting_chair==0):
        barber_sleep=1
        print("The barber has gone to sleep!")


def balk():
    sem_leave.acquire()    #semWait(sem_leave)
    print("%s acquired lock"%(threading.current_thread().name))
    print("semaphore sem_leave value: ",sem_leave._value)
    if(occupied_waiting_chair==0):
       pass
    elif(occupied_waiting_chair==total_waiting_chair):
        time.sleep(1) 
        print("The shop is full, so customer has left the shop without being entertained.")
        
def main():
    
    print("Welcome to Barber's shop!")

    t1 = threading.Thread(target = customer)
    t2 = threading.Thread(target = get_haircut)
    t3 = threading.Thread(target = cut_hair)
    t4 = threading.Thread(target = barber)
    t5 = threading.Thread(target = balk)


    t1.start()
    time.sleep(0.2)
    t2.start()
    time.sleep(0.2)
    t3.start()
    time.sleep(0.2)
    t4.start()
    time.sleep(0.2)
    t5.start()

    t1.join()
    t2.join()
    t3.join()
    t4.join()
    t5.join()
    print("All the Threads are done Exiting")

main()
